# ETL Project

### Group Members:  Daniel Adam Cebula, William Lim and Rohan Chaudhari

### Instructor:  Daniel de Repentigny

### Teaching Assistants:  Anirudh Mungre, Amitoz Ralhan

### Section:  Monday-Wednesday-Saturday

### Goal:

#### For our project we will extract, transform and load several datasets to answer the following question;  For films, video-games and the adaptations in between which came first and how successful where they?

#### An example is the terminator film series and terminator video games.  Pandas .read_html() and .read_csv() will be done on the datasets to generated the Pandas dataframes.

#### From there our group will clean the data and generate processed versions of the dataframes which will be exported to .csv format and it will be imported into a SQL database through the use of PROGRESql on pgAdmin software.

#### Datasets will be taken from the following sources:
<https://data.world/popculture/imdb-5000-movie-dataset>
(generated by a third-party scrape of imdb website)

<https://www.kaggle.com/ashaheedq/video-games-sales-2019/>
(generated by a third-party scrape of vgchartz.com)

<https://en.wikipedia.org/wiki/List_of_films_based_on_video_games>

<https://en.wikipedia.org/wiki/List_of_video_games_based_on_films>

#### The following jupyter notebook details the cleaning of the datasets and the export to a SQL database.

In [1]:
# dependencies to be used
import pandas as pd
import os
import re

In [2]:
# get file paths
parent_path = os.path.realpath("")
raw_input_path = os.path.realpath("Raw_Input")
processed_input_path = os.path.realpath("Processed_Input")

print(f"""\n{parent_path}\n{raw_input_path}\n{processed_input_path}\n""")


C:\Users\danie\Desktop\Project\Final_Project
C:\Users\danie\Desktop\Project\Final_Project\Raw_Input
C:\Users\danie\Desktop\Project\Final_Project\Processed_Input



### Let us first focus on the video game and  movie datsets from data.world and kaggle

In [3]:
# get file paths for movie and video game .csv files
movie_metadata = os.path.join(raw_input_path,\
                              "movie_metadata.csv")
vgsales = os.path.join(raw_input_path,\
                              "vgsales-12-4-2019.csv")

print(f"""\n{movie_metadata}\n{vgsales}\n""")


C:\Users\danie\Desktop\Project\Final_Project\Raw_Input\movie_metadata.csv
C:\Users\danie\Desktop\Project\Final_Project\Raw_Input\vgsales-12-4-2019.csv



In [4]:
# get pandas dataframe from .csv
movie_metadata_df = pd.read_csv(movie_metadata)
vgsales_df = pd.read_csv(vgsales)

In [5]:
# preview movies dataframe
movie_metadata_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [6]:
# preview video games dataframe
vgsales_df.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


In [7]:
print(f"""
Movie Dataset:
Row #: {movie_metadata_df.shape[0]}
Column #: {movie_metadata_df.shape[1]}
Video Game Dataset:
Row #: {vgsales_df.shape[0]}
Column#: {vgsales_df.shape[1]}
""")


Movie Dataset:
Row #: 5043
Column #: 28
Video Game Dataset:
Row #: 55792
Column#: 23



In [8]:
# let's get column names as the preview is not showing
# all column names
for x in movie_metadata_df.columns:
    print(f"Movie Column: {x}")
print("\n")
for y in vgsales_df.columns:
    print(f"Video Game Column: {y}")

Movie Column: color
Movie Column: director_name
Movie Column: num_critic_for_reviews
Movie Column: duration
Movie Column: director_facebook_likes
Movie Column: actor_3_facebook_likes
Movie Column: actor_2_name
Movie Column: actor_1_facebook_likes
Movie Column: gross
Movie Column: genres
Movie Column: actor_1_name
Movie Column: movie_title
Movie Column: num_voted_users
Movie Column: cast_total_facebook_likes
Movie Column: actor_3_name
Movie Column: facenumber_in_poster
Movie Column: plot_keywords
Movie Column: movie_imdb_link
Movie Column: num_user_for_reviews
Movie Column: language
Movie Column: country
Movie Column: content_rating
Movie Column: budget
Movie Column: title_year
Movie Column: actor_2_facebook_likes
Movie Column: imdb_score
Movie Column: aspect_ratio
Movie Column: movie_facebook_likes


Video Game Column: Rank
Video Game Column: Name
Video Game Column: basename
Video Game Column: Genre
Video Game Column: ESRB_Rating
Video Game Column: Platform
Video Game Column: Publisher

In [9]:
# let's remove columns from movies and video games that are
# not pertinent to this ETL project
movie_clean1_df = movie_metadata_df[[\
    "movie_title", "plot_keywords", "language", "country",\
    "title_year", "genres", "duration", "director_name",\
    "imdb_score", "movie_imdb_link", "actor_1_name",\
    "actor_2_name", "actor_3_name", "budget", "gross"]].copy()

video_games_clean1_df = vgsales_df[[\
    "Name", "basename", "Year", "Genre", "Publisher",\
    "Developer", "Platform", "url", "Total_Shipped"]].copy()

In [10]:
# Rename column names for both datasets
movie_clean1_df.columns = [
    "Movie_Title", "Plot_Keywords", "Language", "Country",\
    "Year", "Genres", "Duration_minutes", "Director_Name",\
    "IMDB_Score", "IMDB_Link", "Actor_1",\
    "Actor_2", "Actor_3", "Film_Budget", "Gross_Income"]

video_games_clean1_df.columns = [
    "Video_Game_Name", "Key_Words", "Year", "Genre",\
    "Publisher", "Developer", "Platform_Console",\
    "VGChartz_URL", "Total_Shipped"]

In [11]:
# There are imperfect records, they contain various nulls,
# and need to be dropped as they contain no important
# information.  A list of columns are examined for each.
important_list = ["Movie_Title", "Plot_Keywords", "Year",\
                  "Director_Name", "IMDB_Score", "IMDB_Link",\
                 "Film_Budget", "Gross_Income"]
movie_clean1_df.dropna(subset=important_list,\
                        how="any", inplace=True)

important_list = ["Video_Game_Name", "Key_Words",\
                  "Year", "Publisher", "Developer",\
                  "Platform_Console", "VGChartz_URL"]
video_games_clean1_df.dropna(subset=important_list,\
                            how="any", inplace=True)

In [17]:
# For the movies and video games dataset the year values
# have a trailing decimal point
movie_clean1_df["Year"] = movie_clean1_df["Year"].map('{:.0f}'.format)
video_games_clean1_df["Year"] = video_games_clean1_df["Year"].map('{:.0f}'.format)

In [19]:
# For the movies and video games datasets there exists
# columns which contain multiple terms.  Save it to the
# dataframe as an array value
movie_clean1_df["Genres"] = movie_clean1_df["Genres"].\
                            apply(lambda x: x.split('|'))
movie_clean1_df["Plot_Keywords"] = movie_clean1_df[\
                            "Plot_Keywords"].\
                            apply(lambda y: y.split('|'))

video_games_clean1_df["Key_Words"] = video_games_clean1_df[\
                            "Key_Words"].\
                            apply(lambda x: x.split('-'))

In [22]:
# save the dataframes as .csv files for documentation
# purposes
movie_metadata_path = os.path.join(processed_input_path,\
                               "movie_metadata_clean.csv")
movie_clean1_df.to_csv(movie_metadata_path)

video_game_path = os.path.join(processed_input_path,\
                               "vgsales-12-4-2019_clean.csv")
video_games_clean1_df.to_csv(video_game_path)

### Let us now focus on cleaning the html tables from wikipedia for
<https://en.wikipedia.org/wiki/List_of_films_based_on_video_games>
### And
<https://en.wikipedia.org/wiki/List_of_video_games_based_on_films>
### These will be used as a key for the 2 datasets

In [85]:
# get pandas to read the html for tables from the following urls
url = 'https://en.wikipedia.org/wiki/List_of_films_based_on_video_games'
tables1 = pd.read_html(url)
url = 'https://en.wikipedia.org/wiki/List_of_video_games_based_on_films'
tables2 = pd.read_html(url)

In [68]:
# get the first table for live-action releases
df1 = tables1[0].copy()
df1.columns = ['Title', 'Release_Date', 'World_Wide_Box_Office',\
              'Rotten_Tomatoes', 'Metacritic', 'Distributor',\
              'Original_Game_Publisher']

df1["Type"] = "International-Live-Action"
df1.drop(columns=["Metacritic", "Rotten_Tomatoes"],\
         inplace=True)
df1 = df1[df1.World_Wide_Box_Office != "TBA"]
df1

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Super Mario Bros.,"May 28, 1993","$20,915,465[3]",Buena Vista Pictures Distribution,Nintendo,International-Live-Action
1,Double Dragon,"November 4, 1994","$2,341,309[5]",Gramercy Pictures,Technōs Japan,International-Live-Action
2,Street Fighter,"December 23, 1994","$99,423,521[7]",Universal Pictures,Capcom,International-Live-Action
3,Mortal Kombat,"August 18, 1995","$122,195,920[9]",New Line Cinema,Midway Games,International-Live-Action
4,Mortal Kombat: Annihilation,"November 21, 1997","$51,376,861[12]",New Line Cinema,Midway Games,International-Live-Action
5,Wing Commander,"March 12, 1999","$11,578,059[15]",20th Century Fox,Origin Systems,International-Live-Action
6,Lara Croft: Tomb Raider,"June 15, 2001","$274,703,340[18]",Paramount Pictures,Eidos,International-Live-Action
7,Resident Evil,"March 15, 2002","$102,984,862[21]",Screen Gems,Capcom,International-Live-Action
8,Lara Croft: Tomb Raider – The Cradle of Life,"July 25, 2003","$156,505,388[24]",Paramount Pictures,Eidos,International-Live-Action
9,House of the Dead,"October 10, 2003","$13,818,181[27]",Artisan Entertainment,Sega,International-Live-Action


In [69]:
df1["Title"] = df1["Title"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df1["Release_Date"] = df1["Release_Date"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df1["World_Wide_Box_Office"] = df1["World_Wide_Box_Office"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df1["Distributor"] = df1["Distributor"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df1["Original_Game_Publisher"] = df1["Original_Game_Publisher"].apply(lambda x: re.sub(r'\[.*\]', '', x))

df1

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Super Mario Bros.,"May 28, 1993","$20,915,465",Buena Vista Pictures Distribution,Nintendo,International-Live-Action
1,Double Dragon,"November 4, 1994","$2,341,309",Gramercy Pictures,Technōs Japan,International-Live-Action
2,Street Fighter,"December 23, 1994","$99,423,521",Universal Pictures,Capcom,International-Live-Action
3,Mortal Kombat,"August 18, 1995","$122,195,920",New Line Cinema,Midway Games,International-Live-Action
4,Mortal Kombat: Annihilation,"November 21, 1997","$51,376,861",New Line Cinema,Midway Games,International-Live-Action
5,Wing Commander,"March 12, 1999","$11,578,059",20th Century Fox,Origin Systems,International-Live-Action
6,Lara Croft: Tomb Raider,"June 15, 2001","$274,703,340",Paramount Pictures,Eidos,International-Live-Action
7,Resident Evil,"March 15, 2002","$102,984,862",Screen Gems,Capcom,International-Live-Action
8,Lara Croft: Tomb Raider – The Cradle of Life,"July 25, 2003","$156,505,388",Paramount Pictures,Eidos,International-Live-Action
9,House of the Dead,"October 10, 2003","$13,818,181",Artisan Entertainment,Sega,International-Live-Action


In [70]:
# now for the second table
df2 = tables1[1].copy()
df2.columns = ['Title', 'Release_Date', 'World_Wide_Box_Office',\
              'Rotten_Tomatoes', 'Metacritic', 'Distributor',\
              'Original_Game_Publisher']

df2["Type"] = "Animated"
df2.drop(columns=["Metacritic", "Rotten_Tomatoes"], inplace=True)
df2 = df2[df2.World_Wide_Box_Office != "TBA"]
df2

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Final Fantasy: The Spirits Within,"July 11, 2001","$85,131,830[110]",Columbia Pictures,Square,Animated
1,Ratchet & Clank,"April 29, 2016","$11,821,329[113]",Gramercy Pictures,Sony Interactive Entertainment,Animated
2,The Angry Birds Movie,"May 20, 2016","$352,333,929[116]",Sony Pictures Releasing,Rovio Entertainment,Animated
3,The Angry Birds Movie 2,"August 14, 2019","$49,231,015[119]",Sony Pictures Releasing,Rovio Entertainment,Animated


In [71]:
df2["Title"] = df2["Title"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df2["Release_Date"] = df2["Release_Date"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df2["World_Wide_Box_Office"] = df2["World_Wide_Box_Office"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df2["Distributor"] = df2["Distributor"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df2["Original_Game_Publisher"] = df2["Original_Game_Publisher"].apply(lambda x: re.sub(r'\[.*\]', '', x))

df2

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Final Fantasy: The Spirits Within,"July 11, 2001","$85,131,830",Columbia Pictures,Square,Animated
1,Ratchet & Clank,"April 29, 2016","$11,821,329",Gramercy Pictures,Sony Interactive Entertainment,Animated
2,The Angry Birds Movie,"May 20, 2016","$352,333,929",Sony Pictures Releasing,Rovio Entertainment,Animated
3,The Angry Birds Movie 2,"August 14, 2019","$49,231,015",Sony Pictures Releasing,Rovio Entertainment,Animated


In [72]:
# now for the third table
df3 = tables1[2].copy()
df3.columns = ['Title', 'Release_Date', 'World_Wide_Box_Office',\
              'Rotten_Tomatoes', 'Metacritic', 'Distributor',\
              'Original_Game_Publisher']

df3["Type"] = "Japanese-International-Distribution"
df3.drop(columns=["Metacritic", "Rotten_Tomatoes"], inplace=True)
df3 = df3[df3.World_Wide_Box_Office != "TBA"]
df3

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Street Fighter II: The Animated Movie,"August 6, 1994","$16,000,000[126]",Toei Company,Capcom,Japanese-International-Distribution
1,Pokémon: The First Movie,"July 18, 1998 (JP)November 10, 1999 (NA)","$172,744,662[127]",Toho,Nintendo,Japanese-International-Distribution
2,Pokémon: The Movie 2000,"July 17, 1999 (JP)July 21, 2000 (NA)","$133,949,270[130]",Toho,Nintendo,Japanese-International-Distribution
3,Pokémon 3: The Movie,"July 8, 2000 (JP)April 6, 2001 (NA)","$68,411,275[133]",Toho,Nintendo,Japanese-International-Distribution
4,Pokémon 4Ever,"July 7, 2001 (JP)October 11, 2002 (NA)","$28,023,563[136]",Toho,Nintendo,Japanese-International-Distribution
5,Pokémon Heroes,"July 13, 2002 (JP)May 16, 2003 (NA)","$20,867,919[139]",Toho,Nintendo,Japanese-International-Distribution
6,Pokémon: Zoroark: Master of Illusions,"July 10, 2010","$71,143,529[142]",Toho,Nintendo,Japanese-International-Distribution
7,Yo-kai Watch: The Movie,"December 20, 2014","$80,268,947[b]",Toho,Level-5,Japanese-International-Distribution
8,Yo-Kai Watch: Enma Daiō to Itsutsu no Monogata...,"December 19, 2015","$58,850,969[c]",Toho,Level-5,Japanese-International-Distribution
9,Kingsglaive: Final Fantasy XV,"July 9, 2016 (JP)August 19, 2016 (NA)","$6,550,000[144][145]",Stage 6 Films,Square Enix,Japanese-International-Distribution


In [73]:
df3["Title"] = df3["Title"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df3["Release_Date"] = df3["Release_Date"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df3["World_Wide_Box_Office"] = df3["World_Wide_Box_Office"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df3["Distributor"] = df3["Distributor"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df3["Original_Game_Publisher"] = df3["Original_Game_Publisher"].apply(lambda x: re.sub(r'\[.*\]', '', x))

df3

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Street Fighter II: The Animated Movie,"August 6, 1994","$16,000,000",Toei Company,Capcom,Japanese-International-Distribution
1,Pokémon: The First Movie,"July 18, 1998 (JP)November 10, 1999 (NA)","$172,744,662",Toho,Nintendo,Japanese-International-Distribution
2,Pokémon: The Movie 2000,"July 17, 1999 (JP)July 21, 2000 (NA)","$133,949,270",Toho,Nintendo,Japanese-International-Distribution
3,Pokémon 3: The Movie,"July 8, 2000 (JP)April 6, 2001 (NA)","$68,411,275",Toho,Nintendo,Japanese-International-Distribution
4,Pokémon 4Ever,"July 7, 2001 (JP)October 11, 2002 (NA)","$28,023,563",Toho,Nintendo,Japanese-International-Distribution
5,Pokémon Heroes,"July 13, 2002 (JP)May 16, 2003 (NA)","$20,867,919",Toho,Nintendo,Japanese-International-Distribution
6,Pokémon: Zoroark: Master of Illusions,"July 10, 2010","$71,143,529",Toho,Nintendo,Japanese-International-Distribution
7,Yo-kai Watch: The Movie,"December 20, 2014","$80,268,947",Toho,Level-5,Japanese-International-Distribution
8,Yo-Kai Watch: Enma Daiō to Itsutsu no Monogata...,"December 19, 2015","$58,850,969",Toho,Level-5,Japanese-International-Distribution
9,Kingsglaive: Final Fantasy XV,"July 9, 2016 (JP)August 19, 2016 (NA)","$6,550,000",Stage 6 Films,Square Enix,Japanese-International-Distribution


In [75]:
df_total = df1.append(df2)
df_total = df_total.append(df3)
df_total.reset_index()
df_total

,Title,Release_Date,World_Wide_Box_Office,Distributor,Original_Game_Publisher,Type
0,Super Mario Bros.,"May 28, 1993","$20,915,465",Buena Vista Pictures Distribution,Nintendo,International-Live-Action
1,Double Dragon,"November 4, 1994","$2,341,309",Gramercy Pictures,Technōs Japan,International-Live-Action
2,Street Fighter,"December 23, 1994","$99,423,521",Universal Pictures,Capcom,International-Live-Action
3,Mortal Kombat,"August 18, 1995","$122,195,920",New Line Cinema,Midway Games,International-Live-Action
4,Mortal Kombat: Annihilation,"November 21, 1997","$51,376,861",New Line Cinema,Midway Games,International-Live-Action
5,Wing Commander,"March 12, 1999","$11,578,059",20th Century Fox,Origin Systems,International-Live-Action
6,Lara Croft: Tomb Raider,"June 15, 2001","$274,703,340",Paramount Pictures,Eidos,International-Live-Action
7,Resident Evil,"March 15, 2002","$102,984,862",Screen Gems,Capcom,International-Live-Action
8,Lara Croft: Tomb Raider – The Cradle of Life,"July 25, 2003","$156,505,388",Paramount Pictures,Eidos,International-Live-Action
9,House of the Dead,"October 10, 2003","$13,818,181",Artisan Entertainment,Sega,International-Live-Action


In [76]:
game_to_film_path = os.path.join(processed_input_path,\
                               "game_to_film.csv")
df_total.to_csv(game_to_film_path)

In [86]:
df4 = tables2[0].copy()
df4

,Name,Year,Developer,Publisher,Film
0,3 Ninjas Kick Back (video game),1994,Malibu Interactive,Sony Imagesoft,3 Ninjas Kick Back
1,007: Licence to Kill,1989,Quixel,Domark,Licence to Kill
2,007 Legends,2012,Eurocom,Activision,"Goldfinger, On Her Majesty's Secret Service, M..."
3,102 Dalmatians: Puppies to the Rescue,2000,Digital Eclipse (GBC) Toys for Bob[a],"Eidos Interactive (PS1, PC, DC) Activision (GBC)",102 Dalmatians
4,300: March to Glory,2007,Collision Studios,Warner Bros. Games,300
5,A Sound of Thunder (video game),2004,Möbius Entertainment,BAM! Entertainment,A Sound of Thunder
6,A.D. 2044,1996,R.M.P. Software,LK Avalon,Sexmission
7,Action Man: Robot Atak,2004,Magic Pockets,Atari,Action Man: Robot Atak
8,Addams Family Values (video game),1995,Ocean Software,Ocean Software,Addams Family Values
9,The Addams Family (video game),1992,Ocean Software,Ocean Software,The Addams Family


In [87]:
df4.dropna(inplace=True)
df4

,Name,Year,Developer,Publisher,Film
0,3 Ninjas Kick Back (video game),1994,Malibu Interactive,Sony Imagesoft,3 Ninjas Kick Back
1,007: Licence to Kill,1989,Quixel,Domark,Licence to Kill
2,007 Legends,2012,Eurocom,Activision,"Goldfinger, On Her Majesty's Secret Service, M..."
3,102 Dalmatians: Puppies to the Rescue,2000,Digital Eclipse (GBC) Toys for Bob[a],"Eidos Interactive (PS1, PC, DC) Activision (GBC)",102 Dalmatians
4,300: March to Glory,2007,Collision Studios,Warner Bros. Games,300
5,A Sound of Thunder (video game),2004,Möbius Entertainment,BAM! Entertainment,A Sound of Thunder
6,A.D. 2044,1996,R.M.P. Software,LK Avalon,Sexmission
7,Action Man: Robot Atak,2004,Magic Pockets,Atari,Action Man: Robot Atak
8,Addams Family Values (video game),1995,Ocean Software,Ocean Software,Addams Family Values
9,The Addams Family (video game),1992,Ocean Software,Ocean Software,The Addams Family


In [88]:
df4["Name"] = df4["Name"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df4["Year"] = df4["Year"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df4["Developer"] = df4["Developer"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df4["Publisher"] = df4["Publisher"].apply(lambda x: re.sub(r'\[.*\]', '', x))
df4["Film"] = df4["Film"].apply(lambda x: re.sub(r'\[.*\]', '', x))

df4

,Name,Year,Developer,Publisher,Film
0,3 Ninjas Kick Back (video game),1994,Malibu Interactive,Sony Imagesoft,3 Ninjas Kick Back
1,007: Licence to Kill,1989,Quixel,Domark,Licence to Kill
2,007 Legends,2012,Eurocom,Activision,"Goldfinger, On Her Majesty's Secret Service, M..."
3,102 Dalmatians: Puppies to the Rescue,2000,Digital Eclipse (GBC) Toys for Bob,"Eidos Interactive (PS1, PC, DC) Activision (GBC)",102 Dalmatians
4,300: March to Glory,2007,Collision Studios,Warner Bros. Games,300
5,A Sound of Thunder (video game),2004,Möbius Entertainment,BAM! Entertainment,A Sound of Thunder
6,A.D. 2044,1996,R.M.P. Software,LK Avalon,Sexmission
7,Action Man: Robot Atak,2004,Magic Pockets,Atari,Action Man: Robot Atak
8,Addams Family Values (video game),1995,Ocean Software,Ocean Software,Addams Family Values
9,The Addams Family (video game),1992,Ocean Software,Ocean Software,The Addams Family


In [89]:
df4["Name"] = df4["Name"].apply(lambda x: re.sub("\((.*?)\)", '', x))
df4["Year"] = df4["Year"].apply(lambda x: re.sub("\((.*?)\)", '', x))
df4["Developer"] = df4["Developer"].apply(lambda x: re.sub("\((.*?)\)", '', x))
df4["Publisher"] = df4["Publisher"].apply(lambda x: re.sub("\((.*?)\)", '', x))
df4

,Name,Year,Developer,Publisher,Film
0,3 Ninjas Kick Back,1994,Malibu Interactive,Sony Imagesoft,3 Ninjas Kick Back
1,007: Licence to Kill,1989,Quixel,Domark,Licence to Kill
2,007 Legends,2012,Eurocom,Activision,"Goldfinger, On Her Majesty's Secret Service, M..."
3,102 Dalmatians: Puppies to the Rescue,2000,Digital Eclipse Toys for Bob,Eidos Interactive Activision,102 Dalmatians
4,300: March to Glory,2007,Collision Studios,Warner Bros. Games,300
5,A Sound of Thunder,2004,Möbius Entertainment,BAM! Entertainment,A Sound of Thunder
6,A.D. 2044,1996,R.M.P. Software,LK Avalon,Sexmission
7,Action Man: Robot Atak,2004,Magic Pockets,Atari,Action Man: Robot Atak
8,Addams Family Values,1995,Ocean Software,Ocean Software,Addams Family Values
9,The Addams Family,1992,Ocean Software,Ocean Software,The Addams Family


In [90]:
film_to_game_path = os.path.join(processed_input_path,\
                               "film_to_game.csv")
df4.to_csv(film_to_game_path)